<a href="https://colab.research.google.com/github/jasonfangmagic/ML_Notes/blob/main/Facebook_Robyn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
install.packages("Robyn")

In [ ]:
%%R
install.packages("reticulate")

In [ ]:
%%R
library(reticulate)

In [ ]:
pip install nevergrad

In [ ]:
%%R
data("dt_simulated_weekly")

In [ ]:
%%R
head(dt_simulated_weekly)